### Sorting and visualizing UniPept lowest common ancestor analyses in Trocas7

#### Beginning with: LCA peptides in csvs

#### Goal: spectral abundance-corrected (NAAF) taxonomic peptide compositions at the 4 stations before and after 24hrs

## Issue: the `join` commands at the end for diatom and bacterial peptides need to be run independently from one another. To do this, run the diatom command, restart the kernal, then run the bacterial. 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [2]:
cd /home/millieginty/Documents/git-repos/amazon/analyses/T7-incubations/unipept/

/home/millieginty/Documents/git-repos/amazon/analyses/T7-incubations/unipept


In [3]:
# read the data into pandas dataframes
BY_T00_GF_102a = pd.read_csv("lca/cleaned/102_BY_T00_GF_lca.csv")

# delete any Metazoa hits because they're trypsin
BY_T00_GF_102b = BY_T00_GF_102a[BY_T00_GF_102a.kingdom != 'Metazoa']

# keep only entries to the phylum level
BY_T00_GF_102 = BY_T00_GF_102b[BY_T00_GF_102b['phylum'].notnull()]

# How many peptides to the phylum level?
print('# of phylum peptides = ', len(BY_T00_GF_102))

BY_T00_GF_102.head(6)

# of phylum peptides =  20


,peptide,lca,superkingdom,kingdom,subkingdom,superphylum,phylum,subphylum,superclass,class,...,tribe,subtribe,genus,subgenus,species group,species subgroup,species,subspecies,varietas,forma
6,VATVSPLR,Solirubrobacteraceae,Bacteria,NaN,NaN,NaN,Actinobacteria,NaN,NaN,Thermoleophilia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,VVEVSLPR,Planctomycetales,Bacteria,NaN,NaN,NaN,Planctomycetes,NaN,NaN,Planctomycetia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,LATVLSPR,Gammaproteobacteria,Bacteria,NaN,NaN,NaN,Proteobacteria,NaN,NaN,Gammaproteobacteria,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,VVTADLPR,Chloroflexi,Bacteria,NaN,NaN,NaN,Chloroflexi,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
56,EFHPGLAR,Oleiphilus messinensis,Bacteria,NaN,NaN,NaN,Proteobacteria,NaN,NaN,Gammaproteobacteria,...,Oleiphilus,NaN,NaN,NaN,Oleiphilus messinensis,NaN,NaN,NaN,NaN,NaN
57,VGDLSVFK,Seonamhaeicola,Bacteria,NaN,NaN,NaN,Bacteroidetes,NaN,NaN,Flavobacteriia,...,Seonamhaeicola,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Now I want to read in the file containing the stripped peptides with NAAF values
####  - NAAF stands for 'noramlized area abunace factor'

### I want to join the dataframes if they share an index (stripped peptide with equated leucine and isoleucines)
#### - That means I'll reindex the processed peptide file

In [6]:
BY_T00_GF_102_NAAFa = pd.read_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/I-L_NAAFs/102A_BY_T00_GF_DN50_ILnaafs.csv")
BY_T00_GF_102_NAAFb = pd.read_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/I-L_NAAFs/102B_BY_T00_GF_DN50_ILnaafs.csv")
BY_T00_GF_102_NAAFc = pd.read_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/I-L_NAAFs/102C_BY_T00_GF_DN50_ILnaafs.csv")


frames = [BY_T00_GF_102_NAAFa, BY_T00_GF_102_NAAFb]

BY_T00_GF_102_NAAF = pd.concat(frames, sort=False)

BY_T00_GF_102_NAAF.set_index('stripped_peptide')
BY_T00_GF_102_NAAF = BY_T00_GF_102_NAAF.loc[:, ~BY_T00_GF_102_NAAF.columns.str.contains('^Unnamed')]

BY_T00_GF_102_NAAF.rename(columns = {'stripped_peptide':'peptide'}, inplace = True)

print('# of total peptides = ', len(BY_T00_GF_102_NAAF))

print('column names:', BY_T00_GF_102_NAAF.columns)

BY_T00_GF_102_NAAF.head()

# of total peptides =  431
column names: Index(['peptide', 'Area', 'NAAF_num.'], dtype='object')


,peptide,Area,NAAF_num.
0,LSSPATLNSR,60000000.0,6.000000e+06
1,LSSPATLNSR,60000000.0,6.000000e+06
2,LSSPATLNSR,60000000.0,6.000000e+06
3,RSLQVR,76600.0,1.276667e+04
4,LSSPATLDSR,290000.0,2.900000e+04


In [7]:
# get the intersection of the phylum peptides and corresponding peptides w/ NAAFs

over = np.intersect1d(BY_T00_GF_102["peptide"], BY_T00_GF_102_NAAF["peptide"])

print(over)

['EFHPGLAR' 'LATVLSPR' 'TATVSTLR' 'VATVPLNR' 'VATVSPLR' 'VGDLSVFK'
 'VGHWLPR' 'VVEVSLPR' 'VVTADLPR']


In [8]:
# join the dataframes if the peptide values are the same using 'join'
# since a couple are de novo only (more for bacteria), we won't have all the UniPept peptides overlap 

BY_T00_GF_102.set_index('peptide', inplace=True)
BY_T00_GF_102_NAAF.set_index('peptide', inplace=True)

BY_T00_GF_102_Phy = BY_T00_GF_102.join(BY_T00_GF_102_NAAF, how='left', rsuffix='_other')


print('# of total phylum-level peptides = ', len(BY_T00_GF_102_Phy))

BY_T00_GF_102_Phy.head()

# of total phylum-level peptides =  43


,lca,superkingdom,kingdom,subkingdom,superphylum,phylum,subphylum,superclass,class,subclass,...,genus,subgenus,species group,species subgroup,species,subspecies,varietas,forma,Area,NAAF_num.
peptide,,,,,,,,,,,,,,,,,,,,,
EFHPGLAR,Oleiphilus messinensis,Bacteria,NaN,NaN,NaN,Proteobacteria,NaN,NaN,Gammaproteobacteria,NaN,...,NaN,NaN,Oleiphilus messinensis,NaN,NaN,NaN,NaN,NaN,131.0,16.375
EFHPGLAR,Oleiphilus messinensis,Bacteria,NaN,NaN,NaN,Proteobacteria,NaN,NaN,Gammaproteobacteria,NaN,...,NaN,NaN,Oleiphilus messinensis,NaN,NaN,NaN,NaN,NaN,41400.0,5175.000
EFHPGLAR,Oleiphilus messinensis,Bacteria,NaN,NaN,NaN,Proteobacteria,NaN,NaN,Gammaproteobacteria,NaN,...,NaN,NaN,Oleiphilus messinensis,NaN,NaN,NaN,NaN,NaN,1820000.0,227500.000
EFHPGLAR,Oleiphilus messinensis,Bacteria,NaN,NaN,NaN,Proteobacteria,NaN,NaN,Gammaproteobacteria,NaN,...,NaN,NaN,Oleiphilus messinensis,NaN,NaN,NaN,NaN,NaN,1820000.0,227500.000
GALLADVTK,Deferribacteraceae,Bacteria,NaN,NaN,NaN,Deferribacteres,NaN,NaN,Deferribacteres,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# write to a csv

BY_T00_GF_102_Phy.to_csv("lca/NAAF/BY_T00_GF_102_Phy_naaf.csv")